In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################

from flask import Flask, render_template, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy     # Import the SQLAlchemy module from the flask_sqlalchemy package
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user

app = Flask(__name__)
db = SQLAlchemy(app)                                                # Initialize the SQLAlchemy flask database object
app.config['SQLALCHEMY_DATABASE_URI']  = 'sqlite:///demo.db'        # Setup the database file "demo.db"
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False                # Disable modification track
app.config['SECRET_KEY'] = 'mysecretkey'                            # Initialize SECRET_KEY used to encrypt your cookies 
                                                                    # and save send them to the browser for session management.
db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'

class Users(UserMixin, db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(50))
    email = db.Column(db.String(255))
    password = db.Column(db.String(80))
    
@login_manager.user_loader
def load_user(user_id):
    return Users.query.get(int(user_id))

    
################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

################################################################################################################

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/sample")
def sample():
    return render_template('sample.html')

@app.route('/login', methods=["GET", "POST"])
def login():
    message=''
    status = "login"
    if request.method == "POST":
        username = request.form['username']
        password = request.form['password']
        print(username,password)

        user = Users.query.filter_by(username=username).first()
        if not user:
            print("Wrong username or username not found.")
            flash("Wrong username or username not found.")
            return redirect(url_for('login'))
        else:
            if password == user.password:
                print("Successfully logged in.")
                login_user(user)
                flash("Successfully logged in.")
                return redirect(url_for('profile'))
            else:
                print("Wrong password.")
                flash("Wrong password.")
                return redirect(url_for('login'))           
    else:
        return render_template('login.html',status=status)

@app.route("/signup", methods=["GET", "POST"])
def signup():
    status = "signup"
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        print(username,email,password)
        new_user = Users(username=username,email=email,password=password)     
        try:
            db.session.add(new_user)
            db.session.commit()
            print("Registered!")
            return redirect(url_for('index'))
        except Exception as e:
            print("Exception occurred.",e)
            return 'Fail to add user.'
    else:
        return render_template('signup.html',status=status)

@app.route("/profile")
@login_required
def profile():
    status = "profile"
    return render_template('profile.html',username=current_user.username, status=status)
    
@app.route("/logout")
@login_required
def logout():
    print('logout user.')
    flash('logged out.', current_user.username)
    logout_user()
    return redirect(url_for('index')) 

################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


################################################################################################################

# APPLICATION TEST RUN AT PORT 9003

################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 9087)

 * Serving Flask app '__main__'
 * Debug mode: off


/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/flask_sqlalchemy/__init__.py:847: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  warnings.warn(
/Users/suen/Documents/anaconda3/lib/python3.9/site-packages/flask_sqlalchemy/__init__.py:868: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9087
Press CTRL+C to quit
127.0.0.1 - - [07/Nov/2022 16:36:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2022 16:36:54] "GET /static/assets/css/fontawesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Nov/2022 16:36:54] "GET /static/assets/css/templatemo.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Nov/2022 16:36:54] "GET /static/assets/css/custom.css HTTP/1.1" 304 -
127.0.0.1 - - [07/Nov/2022 16:36:54] "GET /static/